In [1]:
import pandas as pd
import numpy as np

from bokeh.plotting import figure, output_notebook, show
from bokeh.models import Band, ColumnDataSource


output_notebook()

from election_data import ElectionData

Loading BokehJS ...

In [2]:
data_url = 'https://www.wahlrecht.de/umfragen/europawahl.htm'

parties = ['CDU', 'SPD', 'GRÜNE','FDP', 'LINKE' , 'AfD']

data = load_url(data_url, parties)
data = data[data['Date'] > pd.to_datetime('2018-01-01')]


data.head()

,Date,CDU,SPD,GRÜNE,FDP,LINKE,AfD
Idx,,,,,,,
2018-10-25,2018-10-25,27.0,16.0,20.0,8.0,10.0,16.0
2018-12-12,2018-12-12,30.0,16.5,17.5,9.0,9.5,14.5
2019-01-14,2019-01-14,35.0,15.0,20.0,8.0,6.0,10.0
2019-02-01,2019-02-01,30.0,15.0,17.0,8.0,8.0,12.0
2019-02-14,2019-02-14,33.0,18.0,19.0,7.0,6.0,10.0


In [2]:
european_election = ElectionData('https://www.wahlrecht.de/umfragen/europawahl.htm')

In [4]:
for party in parties:
    data[party + '_mean'] = data[party].rolling('30d').mean()
    data[party + '_std'] = data[party].rolling('30d').std()
    data[party + '_low'] = data[party + '_mean'] - data[party + '_std']
    data[party + '_up'] = data[party + '_mean']  data[party + '_std']

In [5]:
def get_data(field):
    return [x for x in data[field]]

In [4]:
p = figure(plot_width=600, plot_height=350, x_axis_type="datetime")

parties = ['CDU', 'SPD', 'GRÜNE','FDP', 'LINKE' , 'AfD']


party_to_color = {
'CDU': 'black',
'SPD': 'red',
'GRÜNE': 'green',
'FDP': 'yellow',
'AfD': 'blue',
'LINKE': 'purple'
}

# source = ColumnDataSource(data.reset_index())
source = european_election.asColumnDataSource()

bands = {}

for party in parties:
    color = party_to_color[party]
    p.circle('Date', party, source=source, size=4, color=color)
    p.line('Date', party + '_mean', source=source, color=color)
    bands[party] = Band(base='Date', lower=party+'_low', upper=party+'_up', source=source, level='underlay', fill_alpha=0.2, line_width=1, fill_color=color)
    p.add_layout(bands[party])

show(p)

In [18]:
source

ColumnDataSource(id='4109', ...)

In [15]:
bands

{'CDU': Band(id='4120', ...),
 'SPD': Band(id='4132', ...),
 'GRÜNE': Band(id='4144', ...),
 'FDP': Band(id='4156', ...),
 'LINKE': Band(id='4168', ...),
 'AfD': Band(id='4180', ...)}